<a href="https://colab.research.google.com/github/DJCordhose/ml-workshop/blob/master/notebooks/process/export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline and Export

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

In [12]:
import sklearn
sklearn.__version__

'0.21.2'

In [0]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/DJCordhose/deep-learning-crash-course-notebooks/master/data/insurance-customers-1500.csv', sep=';')

In [0]:
y=df['group']

In [0]:
df.drop('group', axis='columns', inplace=True)

In [0]:
X = df.as_matrix()

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1200, 3), (1200,), (300, 3), (300,))

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(3,20),
    "max_depth": randint(2, 25),
    "min_samples_split": randint(2, 11),
    "min_samples_leaf": randint(1, 11)
}

clf = RandomizedSearchCV(RandomForestClassifier(), param_dist, n_iter=1000, n_jobs=-1)
%time clf.fit(X_train, y_train)
clf.best_params_

CPU times: user 3.8 s, sys: 95.6 ms, total: 3.89 s
Wall time: 39 s


{'max_depth': 13,
 'min_samples_leaf': 1,
 'min_samples_split': 8,
 'n_estimators': 14}

In [21]:
# parameters found might vary a bit with each run, because it is a random search
clf = RandomForestClassifier(max_depth=13, min_samples_leaf=1, min_samples_split=8, n_estimators=14, n_jobs=-1)
%time clf.fit(X_train, y_train)

CPU times: user 37.4 ms, sys: 5.26 ms, total: 42.6 ms
Wall time: 118 ms


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=13, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=14, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [22]:
clf.score(X_train, y_train)

0.9025

In [30]:
clf.score(X_test, y_test)

0.78

In [0]:
!mkdir models

In [0]:
import pickle

filename = 'models/rf.model'
pickle.dump(clf, open(filename, 'wb'))